In [16]:
import time
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import certifi
import pandas as pd

## Database Connection

In [17]:
# InfluxDB configuration
token = '8Kph_oFbnNqTvgUfERhTJveJpw1A08-HDsnhftEIYj8HRSUJSmKIdoPZdaIbNeIkG4likhDrOtr5FgyF5W03hg=='
org = "Ethan"
bucket = "Test"  # Use "bucket" instead of "database" in influxdb-client
url = "https://us-east-1-1.aws.cloud2.influxdata.com"

# Create InfluxDB client
client = InfluxDBClient(url=url, token=token, org=org, ssl_ca_cert=certifi.where())
write_api = client.write_api(write_options=SYNCHRONOUS)

## Test Data

In [18]:
# Example sensor data (hardcoded)
dht_sensor_data = [
    {"Zone": "Zone 1", "Temp_C": 20.5, "Temp_F": 68.9, "Humidity": 55.0},
    {"Zone": "Zone 2", "Temp_C": 21.0, "Temp_F": 69.8, "Humidity": 60.0},
    {"Zone": "Zone 3", "Temp_C": 19.8, "Temp_F": 67.6, "Humidity": 58.0},
]

water_sensor_data = [
    {"sensor": "water_sensor", "RTD": 0.98, "pH": 6.5, "EC": 1.1},
    {"sensor": "water_sensor", "RTD": 0.95, "pH": 6.6, "EC": 1.2},
    {"sensor": "water_sensor", "RTD": 0.99, "pH": 6.4, "EC": 1.3},
]

## Data Upload Functions

In [19]:
# Function to upload DHT sensor data to InfluxDB
def upload_dht_data(data):
    for entry in data:
        point = (
            Point("dht_sensors")  # Measurement for DHT sensors
            .tag("Zone", entry["Zone"])  # Tag for sensor type
            .field("Temp_C", entry["Temp_C"])  # Temperature in Celsius
            .field("Temp_F", entry["Temp_F"])  # Temperature in Fahrenheit
            .field("Humidity", entry["Humidity"])  # Humidity level
        )
        write_api.write(bucket=bucket, org=org, record=point)
        print(f"Uploaded DHT sensor data from {entry['Zone']}: {entry}")
        time.sleep(1)  # Optional: Delay to simulate real-time data collection

# Function to upload water sensor data to InfluxDB
def upload_water_data(data):
    for entry in data:
        point = (
            Point("water_sensors")  # Measurement for water sensors
            .field("RTD", entry["RTD"])  # RTD value
            .field("pH", entry["pH"])  # pH value
            .field("EC", entry["EC"])  # Electrical conductivity
        )
        write_api.write(bucket=bucket, org=org, record=point)
        print(f"Uploaded water sensor data: {entry}")
        time.sleep(1)  # Optional: Delay to simulate real-time data collection

## Data Upload

In [20]:
# Upload data for DHT sensors
upload_dht_data(dht_sensor_data)

Uploaded DHT sensor data from Zone 1: {'Zone': 'Zone 1', 'Temp_C': 20.5, 'Temp_F': 68.9, 'Humidity': 55.0}
Uploaded DHT sensor data from Zone 2: {'Zone': 'Zone 2', 'Temp_C': 21.0, 'Temp_F': 69.8, 'Humidity': 60.0}
Uploaded DHT sensor data from Zone 3: {'Zone': 'Zone 3', 'Temp_C': 19.8, 'Temp_F': 67.6, 'Humidity': 58.0}


In [21]:
# Upload data for water sensors
upload_water_data(water_sensor_data)

Uploaded water sensor data: {'sensor': 'water_sensor', 'RTD': 0.98, 'pH': 6.5, 'EC': 1.1}
Uploaded water sensor data: {'sensor': 'water_sensor', 'RTD': 0.95, 'pH': 6.6, 'EC': 1.2}
Uploaded water sensor data: {'sensor': 'water_sensor', 'RTD': 0.99, 'pH': 6.4, 'EC': 1.3}


## Sample Queries

Return All DHT Data

In [40]:
# Query
querier = client.query_api()
tables = querier.query(f'''
from(bucket: "{bucket}")
  |> range(start: -1y)
  |> filter(fn: (r) => r["_measurement"] == "dht_sensors")''',  #DHT Data 
  org=org)

# Data Extraction
records = []
for table in tables:
    for record in table.records:
        records.append({
            'time': record.get_time(),
            'sensor': record.values['sensor'],
            record.get_field(): record.get_value()
        })

# Convert to Pandas DataFrame
df = pd.DataFrame(records)
df = df.pivot_table(index=['time', 'sensor'], values=['Temp_C', 'Temp_F', 'Humidity'], aggfunc='first').reset_index() # Convert to fields
df['time'] = pd.to_datetime(df['time']).dt.tz_convert('US/Eastern') # Adjust 'time' to EST

df

,time,sensor,Humidity,Temp_C,Temp_F
0,2024-09-09 14:41:03.937043-04:00,DHT17,55.0,20.5,68.9
1,2024-09-09 14:41:05.114432-04:00,DHT18,60.0,21.0,69.8
2,2024-09-09 14:41:06.189103-04:00,DHT22,58.0,19.8,67.6


Return All Water Data

In [38]:
# Query
querier = client.query_api()
tables = querier.query(f'''
from(bucket: "{bucket}")
  |> range(start: -1y)
  |> filter(fn: (r) => r["_measurement"] == "water_sensors")''',  #Water data
  org=org)

# Data Extraction
records = []
for table in tables:
    for record in table.records:
        records.append({
            'time': record.get_time(),
            'sensor': record.values['sensor'],
            record.get_field(): record.get_value()
        })

# Convert to Pandas DataFrame
df = pd.DataFrame(records)
df = df.pivot_table(index=['time', 'sensor'], values=['EC', 'pH', 'RTD'], aggfunc='first').reset_index() # Convert to fields
df['time'] = pd.to_datetime(df['time']).dt.tz_convert('US/Eastern') # Adjust 'time' to EST

df

,time,sensor,EC,RTD,pH
0,2024-09-09 14:41:07.772529-04:00,water_sensor,1.1,0.98,6.5
1,2024-09-09 14:41:08.943115-04:00,water_sensor,1.2,0.95,6.6
2,2024-09-09 14:41:10.043278-04:00,water_sensor,1.3,0.99,6.4


Return Specific DHT Data

In [39]:
# Query
querier = client.query_api()
tables = querier.query(f'''
from(bucket: "{bucket}")
  |> range(start: -1y)
  |> filter(fn: (r) => r["_measurement"] == "dht_sensors")
  |> filter(fn: (r) => r["sensor"] == "DHT17")''',  # DHT17 Data
  org=org)

# Data Extraction
records = []
for table in tables:
    for record in table.records:
        records.append({
            'time': record.get_time(),
            'sensor': record.values['sensor'],
            record.get_field(): record.get_value()
        })

# Convert to Pandas DataFrame
df = pd.DataFrame(records)
df = df.pivot_table(index=['time', 'sensor'], values=['Temp_C', 'Temp_F', 'Humidity'], aggfunc='first').reset_index() # Convert to fields
df['time'] = pd.to_datetime(df['time']).dt.tz_convert('US/Eastern') # Adjust 'time' to EST

df

,time,sensor,Humidity,Temp_C,Temp_F
0,2024-09-09 14:41:03.937043-04:00,DHT17,55.0,20.5,68.9
